In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler


<class 'ModuleNotFoundError'>: No module named 'keras'

In [ ]:
# Load the dataset
data = pd.read_csv('time_series_data.csv')


In [ ]:
# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort the dataframe by date
data = data.sort_values('Date')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Value'].values.reshape(-1,1))


In [ ]:
# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]


In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

# Choose sequence length
seq_length = 10

# Create sequences for training and testing
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print model summary
print(model.summary())


In [ ]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=32)


In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions and actual values
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform([y_test])


In [ ]:
# Evaluate the model
mae = mean_absolute_error(y_test[0], predictions[:,0])
rmse = np.sqrt(mean_squared_error(y_test[0], predictions[:,0]))
print('Mean Absolute Error:', mae)
print('Root Mean Squared Error:', rmse)


In [4]:
# Plot actual vs. predicted
plt.plot(y_test[0], label='Actual')
plt.plot(predictions[:,0], label='Predicted')
plt.legend()
plt.show()


<class 'NameError'>: name 'plt' is not defined